# NRP Tutorial: Multiple Robots

Welcome to this turorial. By following this notebook step by step, you will learn how to work with the Neurorobotics Platform.

The following notebook will guide you through setting up an experiment interactively. The experiment involves processing sensory inputs, camera images and laser scans, that will feed small neural networks. Networks responses will be turned into motor commands so as to get some desired behaviours, namely color recognition and obstacle avoidance.

# Get started

Start the NRP with the following commands:
```shell
cle-nginx
cle-start
cle-frontend
```
and launch the experiment **multiple Robots Tutorial**.
The experiment contains an iCub Robot (humanoid robot) and two Pioneer 3DX robots (two-wheeled robots) in an empty world. 

Click the play button. You should see two motionless Pioneer 3DX robots and a still iCub robot.

Press the eye icon of the toolbar: you should see what the iCub sees.

# Exercise 1: Color recognition

The goal of this exercise is to understand the basics of connecting a brain model to a robot model.
At the end, the iCub robot will be able to recognize the color of the robots turning around him and to react accordingly.


## Transfer Functions editor - basic control of the P3DX robots 

We will bring the Pioneer 3DX robots into life by means of two basic controllers implemented as Transfer Functions.

Open the **NRP editors** and select the **Transfer Functions** tab.
Every function defined here are called at regular 20ms intervals.

Press the **New** button to create a new Transfer Function. Remove the content of the default function and paste the following snippet

In [ ]:
from hbp_nrp_cle.robotsim.RobotInterface import Topic
import geometry_msgs.msg

@nrp.Neuron2Robot(Topic('/red_pioneer3dx/cmd_vel', geometry_msgs.msg.Twist))
def turn_around_red_p3dx(t):
    return geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(1.0,0,0),
                                   angular=geometry_msgs.msg.Vector3(0,0,0.25))

Repeat the same operation with this other snippet

In [ ]:
from hbp_nrp_cle.robotsim.RobotInterface import Topic
import geometry_msgs.msg

@nrp.Neuron2Robot(Topic('/green_pioneer3dx/cmd_vel', geometry_msgs.msg.Twist))
def turn_around_green_p3dx(t):
    return geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(1.0,0,0),
                                   angular=geometry_msgs.msg.Vector3(0,0,0.25))

Press the **Apply** button of each Transfer Function and press the play button if needed.

## Brain editor - read how the iCub brain is defined 
Open the **NRP editors** again and select the **Brain** tab. 

The PyNN script you read defines neuron populations.
In this experiment, the iCub brain has only 4 neurons separated. The first two neurons, that is icub_input_left and icub_input_right, are the *sensor neurons*. We will provide them with stimuli based on the camera images. 
Each is connected to a *motor neuron* whose membrane voltage will be used to generate arms motion.


## Transfer Functions editor - shaping sensory input for the iCub network
Open the **NRP editors** again and select the **Transfer Functions** tab.
Press the **New** button to create a new Transfer Function. Remove the content of the default function and paste the following snippet

In [ ]:
import hbp_nrp_cle.tf_framework as nrp

@nrp.MapRobotSubscriber("camera", Topic('/icub_model/left_eye_camera/image_raw', sensor_msgs.msg.Image))
@nrp.MapSpikeSource("left_input_neuron", nrp.brain.icub_input_left[0], nrp.poisson)
@nrp.MapSpikeSource("right_input_neuron", nrp.brain.icub_input_right[0], nrp.poisson)
@nrp.Robot2Neuron()
def eye_sensor_transmit(t, camera, left_input_neuron, right_input_neuron):
    """
    This transfer function uses OpenCV to compute the amount of red and green pixels
    seen by the iCub robot. Then, it maps these numbers
    (see decorators) to the neural network neurons using a Poisson generator.
    """
    bridge = CvBridge()
    red_pixels = green_pixels = 0.0
    if not isinstance(camera.value, type(None)):

        # Boundary limits of what we consider red (in HSV format)
        lower_red = np.array([0, 30, 30])
        upper_red = np.array([0, 255, 255])
        # Boundary limits of what we consider green (in HSV format)
        lower_green = np.array([55, 30, 30])
        upper_green = np.array([65, 255, 255])

        # Get an OpenCV image
        cv_image = bridge.imgmsg_to_cv2(camera.value, "rgb8")

        # Transform image to HSV (easier to detect colors).
        hsv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2HSV)

        # Create a mask where every non red pixel will be a zero.
        red_mask = cv2.inRange(hsv_image, lower_red, upper_red)
        green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
        image_size = (cv_image.shape[0] * cv_image.shape[1])

        if (image_size > 0):
            # Get the number of red and green pixels in the image.
            red_pixels = cv2.countNonZero(red_mask)
            green_pixels = cv2.countNonZero(green_mask)
            
            # We magnify the pixel numbers so as to 
            # create enough spikes for motor response
            magnitude = 0.25
            left_input_neuron.rate = magnitude * red_pixels
            right_input_neuron.rate = magnitude * green_pixels
                


Let us have a close look at image_processing.py.

The camera image of the iCub is processed to extract color components. These color components, that is the amount of red and green pixels, are turned into firing rates. Thus the sensor neurons will spike whenever a sufficiently large part of the image is red or green.

## Brain Visualizer and Spike Monitor: watch iCub's neural activity

Open the **Brain Visualizer** (*brain icon* in the toolbar) to display the whole network in 3D.
Unselect alls populations except iCub ones.

Open the **Spike Monitor** (*bar code icon* in the toolbar) to display spike trains.

Change the synaptic weights and see how spike rates are impacted.

Don't forget to click the **Apply** button and **start** the experiment if necessary.

Change the **magnitude** factor at the bottom of the script, and see how the network behaves.


## Transfer Functions editor - Actuating iCub arms 

We are going to bring the iCub into life. The following snippet turns membrane voltages of iCub's output neurons into
motor commands. The iCub will raise the left arm whenever he sees a red robot, the right arm whenever he sees a green robot.

Open the **NRP editors** again and select the **Transfer Functions** tab. Create a new Transfer Function by pressing the **New** button. Erase the default python code and replace it by the following snippet.

In [ ]:
import hbp_nrp_cle.tf_framework as nrp
from hbp_nrp_cle.robotsim.RobotInterface import Topic
import geometry_msgs.msg

@nrp.MapSpikeSink("left_output_neuron", nrp.brain.icub_output_left[0], nrp.leaky_integrator_alpha)
@nrp.MapSpikeSink("right_output_neuron", nrp.brain.icub_output_right[0], nrp.leaky_integrator_alpha)
@nrp.MapRobotPublisher('l_shoulder_roll', Topic('/robot/l_shoulder_roll/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('l_shoulder_pitch', Topic('/robot/l_shoulder_pitch/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('l_shoulder_yaw', Topic('/robot/l_shoulder_yaw/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('l_elbow', Topic('/robot/l_elbow/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('r_shoulder_roll', Topic('/robot/r_shoulder_roll/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('r_shoulder_pitch', Topic('/robot/r_shoulder_pitch/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('r_shoulder_yaw', Topic('/robot/r_shoulder_yaw/pos', std_msgs.msg.Float64))
@nrp.MapRobotPublisher('r_elbow', Topic('/robot/r_elbow/pos', std_msgs.msg.Float64))
@nrp.MapVariable("left_leak", initial_value=1)
@nrp.MapVariable("right_leak", initial_value=1)
@nrp.Neuron2Robot()
def icub_control(t, left_output_neuron, right_output_neuron,
    l_shoulder_roll, l_shoulder_pitch, l_shoulder_yaw, l_elbow, 
    r_shoulder_roll, r_shoulder_pitch, r_shoulder_yaw, r_elbow, left_leak, right_leak):
    
    def wave_hand(side, roll, pitch, yaw, elbow):
        if side == 1:
            r_shoulder_roll.send_message(std_msgs.msg.Float64(roll))
            r_shoulder_pitch.send_message(std_msgs.msg.Float64(pitch))
            r_shoulder_yaw.send_message(std_msgs.msg.Float64(yaw))
            r_elbow.send_message(std_msgs.msg.Float64(elbow))
        elif side == -1:
            l_shoulder_roll.send_message(std_msgs.msg.Float64(roll))
            l_shoulder_pitch.send_message(std_msgs.msg.Float64(pitch))
            l_shoulder_yaw.send_message(std_msgs.msg.Float64(yaw))
            l_elbow.send_message(std_msgs.msg.Float64(elbow))

    def actuate_arm(side, voltage, leak):
        elbow = 1.5
        threshold = 0.02
        if voltage > threshold:
            wave_hand(side, 1.7, -1.1, 0., elbow)
            return 0.0
        else:
            leak_inc = 0.007
            leak = leak + leak_inc
            if leak > 1.0:
                leak = 1.0
            r = 1 - leak
            wave_hand(side, 1.7 * r, -1.1 * r, 0., elbow)
            return leak
    
    left_leak.value = actuate_arm(-1, left_output_neuron.voltage, left_leak.value)
    right_leak.value = actuate_arm(1, right_output_neuron.voltage, right_leak.value)


Press the **Apply** button, and also the play button if needed.

Change the **threshold** value. How does it affect the iCub behaviour?

Change the **leak** increment **leak_inc** to 0.01, then to 0.001. How does it impact the arm waving motion? 

# Exercise 2: Obstacle avoidance based on laser scans

The goal of this exercise is to implement an obstacle avoidance behaviour for the red Pioneer 3DX robot. We will use the laser scans of the onboard Hoyuko device to feed the sensor neurons of the Pioneer 3DX brain model.

## Transfer Functions Editor: collect and handle laser scans

Open the **NRP editors** again and select the **Transfer Functions** tab. Create a new Transfer Function by pressing the **New** button. Erase the default python code and replace it by the following snippet.

In [ ]:
@nrp.MapRobotSubscriber("laser", Topic('/p3dx_red/laser/scan', sensor_msgs.msg.LaserScan))
@nrp.MapSpikeSource("red_laser_on", nrp.map_neurons(range(0, 45), lambda i: nrp.brain.red_laser_on[i]), nrp.dc_source)
@nrp.MapSpikeSource("red_laser_off", nrp.map_neurons(range(0, 45), lambda i: nrp.brain.red_laser_off[i]), nrp.dc_source)
@nrp.Robot2Neuron()
def read_red_laser(t, laser, red_laser_on, red_laser_off):
    if laser.value is not None:
        weights = [x != float('inf') and x < 0.5 for x in laser.value.ranges]
        for i in xrange(45):
            red_laser_on[i].amplitude = 0.5 * weights[8*i:8*(i+1)].count(True)
            red_laser_off[i].amplitude = 0.5 * weights[8*i:8*(i+1)].count(False)

Press the **Apply** button, and also the play button if needed.

## Brain Visualizer and Spike Monitor: watch Pioneer 3DX's neural activity

Open the **Brain Visualizer** by pressing on the *brain icon* on the toolbar. Unselect the iCub populations.
Open the **Spike Monitor** by pressing on the *bar code icon*. 

Pause the simulation and put your favorite object in front of the red Pioneer 3DX. To do so, open the **NRP editors** and select the **Environment editor** tab.

Then watch the neural activity in the above visualization widgets.

## Transfer Functions Editor: enabling obstacle avoidance on the Pioneer 3DX

Open the **NRP editors** again and select the **Transfer Functions** tab. Create a new Transfer Function by pressing the **New** button. Erase the default python code and replace it by the following snippet.

In [5]:
from hbp_nrp_cle.robotsim.RobotInterface import Topic
import geometry_msgs.msg

@nrp.MapSpikeSink("red_fw", nrp.brain.red_fw[0], nrp.leaky_integrator_alpha)
@nrp.MapSpikeSink("red_bw", nrp.brain.red_bw[0], nrp.leaky_integrator_alpha)
@nrp.MapSpikeSink("red_left", nrp.brain.red_left[0], nrp.leaky_integrator_alpha)
@nrp.MapSpikeSink("red_right", nrp.brain.red_right[0], nrp.leaky_integrator_alpha)
@nrp.Neuron2Robot(Topic('/red_pioneer3dx/cmd_vel', geometry_msgs.msg.Twist))
def turn_around_red_p3dx(t, red_fw, red_bw, red_left, red_right):
    return geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(15 * red_fw.voltage if red_bw.voltage < 0.03 else  -8 * red_bw.voltage,0,0),
                                       angular=geometry_msgs.msg.Vector3(0,0, (100 * (red_right.voltage - red_left.voltage)) if (red_right.voltage > 0.5 or red_left.voltage > 0.5) else 0.02))

Remove the former turn_around_red.py and turn_around_green.py Transfer Functions.
Press the **Apply** button, and also the play button if needed.
Pause the simulation and put an object in front of the red Pioneer 3DX. 
Press play and observe the behaviour of the red Pioneer 3DX. 
Open the **Brain Visualizer** and the **Spike Monitor** to monitor brain activity while the Pioneer robot is performing is task.

## Replicate the obstacle avoidance algorithm for the red Pioneer 3DX

Create new Transfer Functions for the green Pioneer 3DX robot so that it behaves just like the red one.
You may need to open the **Brain Editor** to get the corresponding population names.